### Part 1: Building the Core Architecture

This notebook covers the main part of the project: implementing the custom Swin Transformer model. The first step is to define all the custom layers needed, like PatchEmbedding, WindowAttention, and the main SwinTransformer block itself. This is the foundation of the model.

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import keras
from keras import layers
from keras import ops
import os

# --- Model Hyperparameters ---
num_classes = 5
input_shape = (224, 224, 3)
patch_size = (4, 4)
num_heads = 6
embed_dim = 96
num_mlp = 384
qkv_bias = True
window_size = 7
shift_size = 3
image_dimension = 224

# --- Calculate patch details ---
num_patch_x = input_shape[0] // patch_size[0]
num_patch_y = input_shape[1] // patch_size[1]
print(f"Number of patches: {num_patch_x}x{num_patch_y} = {num_patch_x * num_patch_y}")

Number of patches: 56x56 = 3136


### Helper Functions

These two functions are used to partition the image into windows and then reverse the proccess. This is a core idea of the Swin Transformer to compute attention locally.

In [3]:
def window_partition(x, window_size):
    _, height, width, channels = x.shape
    patch_num_y = height // window_size
    patch_num_x = width // window_size
    x = ops.reshape(
        x, (-1, patch_num_y, window_size, patch_num_x, window_size, channels)
    )
    x = ops.transpose(x, (0, 1, 3, 2, 4, 5))
    windows = ops.reshape(x, (-1, window_size, window_size, channels))
    return windows

def window_reverse(windows, window_size, height, width, channels):
    patch_num_y = height // window_size
    patch_num_x = width // window_size
    x = ops.reshape(
        windows, (-1, patch_num_y, patch_num_x, window_size, window_size, channels)
    )
    x = ops.transpose(x, (0, 1, 3, 2, 4, 5))
    x = ops.reshape(x, (-1, height, width, channels))
    return x

### Core Layers Implementation

Now, I'm defining the main custom layers for the model.

1.  **WindowAttention**: Computes self-attention within local windows.
2.  **SwinTransformer**: The main block that combines window attention with the MLP.
3.  **PatchEmbedding**: Converts image patches to vector embeddings.
4.  **PatchMerging**: Downsamples the image by merging patches, which creates the hirearchical structure.

In [4]:
class WindowAttention(layers.Layer):
    def __init__(self, dim, window_size, num_heads, qkv_bias=True, dropout_rate=0.0, **kwargs):
        super().__init__(**kwargs)
        self.dim = dim
        self.window_size = window_size
        self.num_heads = num_heads
        self.scale = (dim // num_heads) ** -0.5
        self.qkv = layers.Dense(dim * 3, use_bias=qkv_bias)
        self.dropout = layers.Dropout(dropout_rate)
        self.proj = layers.Dense(dim)

        # Define relative position bias
        num_window_elements = (2 * self.window_size[0] - 1) * (2 * self.window_size[1] - 1)
        self.relative_position_bias_table = self.add_weight(
            shape=(num_window_elements, self.num_heads),
            initializer=keras.initializers.Zeros(),
            trainable=True,
        )
        coords_h = np.arange(self.window_size[0])
        coords_w = np.arange(self.window_size[1])
        coords_matrix = np.meshgrid(coords_h, coords_w, indexing="ij")
        coords = np.stack(coords_matrix)
        coords_flatten = coords.reshape(2, -1)
        relative_coords = coords_flatten[:, :, None] - coords_flatten[:, None, :]
        relative_coords = relative_coords.transpose([1, 2, 0])
        relative_coords[:, :, 0] += self.window_size[0] - 1
        relative_coords[:, :, 1] += self.window_size[1] - 1
        relative_coords[:, :, 0] *= 2 * self.window_size[1] - 1
        relative_position_index = relative_coords.sum(-1)
        self.relative_position_index = keras.Variable(
            initializer=relative_position_index,
            shape=relative_position_index.shape,
            dtype="int",
            trainable=False,
        )

    def call(self, x, mask=None):
        _, size, channels = x.shape
        head_dim = channels // self.num_heads
        x_qkv = self.qkv(x)
        x_qkv = ops.reshape(x_qkv, (-1, size, 3, self.num_heads, head_dim))
        x_qkv = ops.transpose(x_qkv, (2, 0, 3, 1, 4))
        q, k, v = x_qkv[0], x_qkv[1], x_qkv[2]
        q = q * self.scale
        k = ops.transpose(k, (0, 1, 3, 2))
        attn = q @ k

        num_window_elements = self.window_size[0] * self.window_size[1]
        relative_position_index_flat = ops.reshape(self.relative_position_index, (-1,))
        relative_position_bias = ops.take(self.relative_position_bias_table, relative_position_index_flat, axis=0)
        relative_position_bias = ops.reshape(relative_position_bias, (num_window_elements, num_window_elements, -1))
        relative_position_bias = ops.transpose(relative_position_bias, (2, 0, 1))
        attn = attn + ops.expand_dims(relative_position_bias, axis=0)

        if mask is not None:
            nW = mask.shape[0]
            mask_float = ops.cast(ops.expand_dims(ops.expand_dims(mask, axis=1), axis=0), "float32")
            attn = ops.reshape(attn, (-1, nW, self.num_heads, size, size)) + mask_float
            attn = ops.reshape(attn, (-1, self.num_heads, size, size))
            attn = keras.activations.softmax(attn, axis=-1)
        else:
            attn = keras.activations.softmax(attn, axis=-1)

        attn = self.dropout(attn)
        x_qkv = attn @ v
        x_qkv = ops.transpose(x_qkv, (0, 2, 1, 3))
        x_qkv = ops.reshape(x_qkv, (-1, size, channels))
        x_qkv = self.proj(x_qkv)
        x_qkv = self.dropout(x_qkv)
        return x_qkv

class SwinTransformer(layers.Layer):
    def __init__(self, dim, num_patch, num_heads, window_size=7, shift_size=0, num_mlp=1024, qkv_bias=True, dropout_rate=0.0, **kwargs):
        super().__init__(**kwargs)
        self.dim = dim
        self.num_patch = num_patch
        self.num_heads = num_heads
        self.window_size = window_size
        self.shift_size = shift_size
        self.num_mlp = num_mlp
        self.norm1 = layers.LayerNormalization(epsilon=1e-5)
        self.attn = WindowAttention(
            dim,
            window_size=(self.window_size, self.window_size),
            num_heads=num_heads,
            qkv_bias=qkv_bias,
            dropout_rate=dropout_rate,
        )
        self.drop_path = layers.Dropout(dropout_rate)
        self.norm2 = layers.LayerNormalization(epsilon=1e-5)
        self.mlp = keras.Sequential([
            layers.Dense(num_mlp),
            layers.Activation(keras.activations.gelu),
            layers.Dropout(dropout_rate),
            layers.Dense(dim),
            layers.Dropout(dropout_rate),
        ])
        if min(self.num_patch) < self.window_size:
            self.shift_size = 0
            self.window_size = min(self.num_patch)

    def build(self, input_shape):
        if self.shift_size > 0:
            height, width = self.num_patch
            h_slices = (slice(0, -self.window_size), slice(-self.window_size, -self.shift_size), slice(-self.shift_size, None))
            w_slices = (slice(0, -self.window_size), slice(-self.window_size, -self.shift_size), slice(-self.shift_size, None))
            mask_array = np.zeros((1, height, width, 1))
            count = 0
            for h in h_slices:
                for w in w_slices:
                    mask_array[:, h, w, :] = count
                    count += 1
            mask_array = ops.convert_to_tensor(mask_array)
            mask_windows = window_partition(mask_array, self.window_size)
            mask_windows = ops.reshape(mask_windows, [-1, self.window_size * self.window_size])
            attn_mask = ops.expand_dims(mask_windows, axis=1) - ops.expand_dims(mask_windows, axis=2)
            attn_mask = ops.where(attn_mask != 0, -100.0, attn_mask)
            attn_mask = ops.where(attn_mask == 0, 0.0, attn_mask)
            self.attn_mask = keras.Variable(initializer=attn_mask, shape=attn_mask.shape, dtype=attn_mask.dtype, trainable=False)
        else:
            self.attn_mask = None

    def call(self, x, training=False):
        height, width = self.num_patch
        _, num_patches_before, channels = x.shape
        x_skip = x
        x = self.norm1(x)
        x = ops.reshape(x, (-1, height, width, channels))
        if self.shift_size > 0:
            shifted_x = ops.roll(x, shift=[-self.shift_size, -self.shift_size], axis=[1, 2])
        else:
            shifted_x = x
        x_windows = window_partition(shifted_x, self.window_size)
        x_windows = ops.reshape(x_windows, (-1, self.window_size * self.window_size, channels))
        attn_windows = self.attn(x_windows, mask=self.attn_mask)
        attn_windows = ops.reshape(attn_windows, (-1, self.window_size, self.window_size, channels))
        shifted_x = window_reverse(attn_windows, self.window_size, height, width, channels)
        if self.shift_size > 0:
            x = ops.roll(shifted_x, shift=[self.shift_size, self.shift_size], axis=[1, 2])
        else:
            x = shifted_x
        x = ops.reshape(x, (-1, height * width, channels))
        x = self.drop_path(x, training=training)
        x = x_skip + x
        x_skip = x
        x = self.norm2(x)
        x = self.mlp(x)
        x = self.drop_path(x)
        x = x_skip + x
        return x

class PatchEmbedding(layers.Layer):
    def __init__(self, num_patch, embed_dim, **kwargs):
        super().__init__(**kwargs)
        self.num_patch = num_patch
        self.proj = layers.Dense(embed_dim)
        self.pos_embed = layers.Embedding(input_dim=num_patch, output_dim=embed_dim)

    def call(self, patch):
        pos = ops.arange(start=0, stop=self.num_patch)
        return self.proj(patch) + self.pos_embed(pos)

class PatchMerging(keras.layers.Layer):
    def __init__(self, num_patch, embed_dim, **kwargs):
        super().__init__(**kwargs)
        self.num_patch = num_patch
        self.embed_dim = embed_dim
        self.linear_trans = layers.Dense(2 * embed_dim, use_bias=False)

    def call(self, x):
        height, width = self.num_patch
        _, _, C = x.shape
        x = ops.reshape(x, (-1, height, width, C))
        x0 = x[:, 0::2, 0::2, :]
        x1 = x[:, 1::2, 0::2, :]
        x2 = x[:, 0::2, 1::2, :]
        x3 = x[:, 1::2, 1::2, :]
        x = ops.concatenate((x0, x1, x2, x3), axis=-1)
        x = ops.reshape(x, (-1, (height // 2) * (width // 2), 4 * C))
        return self.linear_trans(x)

### Model Assembly

Now I will assemble the layers together to build the final model. For this first attempt, I'll use a simple structure with two SwinTransformer blocks followed by a patch merging layer and a final classification head.

In [5]:
# --- Model Assembly ---
patch_feature_dim = patch_size[0] * patch_size[1] * input_shape[2]
num_patches_total = num_patch_x * num_patch_y
input_layer_shape = (num_patches_total, patch_feature_dim)

input_tensor = layers.Input(shape=input_layer_shape)

# Stage 1
x = PatchEmbedding(num_patches_total, embed_dim)(input_tensor)
x = SwinTransformer(
    dim=embed_dim,
    num_patch=(num_patch_x, num_patch_y),
    num_heads=num_heads,
    window_size=window_size,
    shift_size=0,
    num_mlp=num_mlp,
    name="swin_stage1_block1"
)(x)
x = SwinTransformer(
    dim=embed_dim,
    num_patch=(num_patch_x, num_patch_y),
    num_heads=num_heads,
    window_size=window_size,
    shift_size=shift_size,
    num_mlp=num_mlp,
    name="swin_stage1_block2"
)(x)

# Downsampling
x = PatchMerging((num_patch_x, num_patch_y), embed_dim=embed_dim)(x)

# Final Classification Head
x = layers.LayerNormalization(epsilon=1e-5)(x)
x = layers.GlobalAveragePooling1D()(x)
output = layers.Dense(num_classes, activation="softmax")(x)

# Create the final model
model = keras.Model(input_tensor, output)

model.summary()

Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 3136, 48)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ patch_embedding                 │ (None, 3136, 96)       │       305,760 │
│ (PatchEmbedding)                │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ swin_stage1_block1              │ (None, 3136, 96)       │       115,255 │
│ (SwinTransformer)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ swin_stage1_block2              │ (None, 3136, 96)       │       268,919 │
│ (SwinTransformer)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ patch_merging (PatchMerging)    │ (None, 784, 192)       │        73,728 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ layer_normalization_4           │ (None, 784, 192)       │           384 │
│ (LayerNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling1d        │ (None, 192)            │             0 │
│ (GlobalAveragePooling1D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 5)              │           965 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 765,011 (3.52 MB)

 Trainable params: 606,545 (2.31 MB)

 Non-trainable params: 158,466 (1.21 MB)

### Part 2: Data Loading and Initial Training Attempt

With the model architecture defined, the next step is to load the pre-processed data and create the `tf.data` pipeline. This involves loading the NumPy arrays we saved earlier and setting up the data pipeline for training.

After that, I will compile the model and run the first training experiment to get a baseline performance.

In [8]:
# --- Training Hyperparameters ---
learning_rate = 1e-3
batch_size = 64
num_epochs = 150
weight_decay = 0.0001
label_smoothing = 0.1

# --- Load Pre-processed Data ---
# Note: This assumes Google Drive is mounted.
try:
    from google.colab import drive
    drive.mount('/content/drive')
    DRIVE_LOAD_DIR = "/content/drive/MyDrive/Colab_Datasets/APTOS_Processed_Augmented"
    x_train = np.load(os.path.join(DRIVE_LOAD_DIR, 'X_train.npy'))
    y_train = np.load(os.path.join(DRIVE_LOAD_DIR, 'y_train_one_hot.npy'))
    # For now, I will create a validation set from the training data for a quick check
    print(f"Loaded x_train shape: {x_train.shape} - y_train shape: {y_train.shape}")
except Exception as e:
    print(f"Error loading data. Make sure you've run the preprocessing notebook. Details: {e}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Loaded x_train shape: (8790, 224, 224, 3) - y_train shape: (8790, 5)


### Create `tf.data` Pipeline

I'm creating the data pipeline. One important step here is extracting patches from the images, because the Swin Transformer doesn't work on whole images directly. The `patch_extract_wrapper` function handles this.

In [9]:
# This function extracts patches from images, a required step for the model
def patch_extract_wrapper(images):
    batch_size = tf.shape(images)[0]
    patches = tf.image.extract_patches(
        images=images,
        sizes=(1, patch_size[0], patch_size[1], 1),
        strides=(1, patch_size[0], patch_size[1], 1),
        rates=(1, 1, 1, 1),
        padding="VALID",
    )
    patch_dim = patches.shape[-1]
    patch_num = patches.shape[1]
    return tf.reshape(patches, (batch_size, patch_num * patch_num, patch_dim))

# Creating a validation split from the training data for this run
val_split_index = int(len(x_train) * 0.9)
x_val_split = x_train[val_split_index:]
y_val_split = y_train[val_split_index:]
x_train_split = x_train[:val_split_index]
y_train_split = y_train[:val_split_index]


# Create the training dataset pipeline
dataset = (
    tf.data.Dataset.from_tensor_slices((x_train_split, y_train_split))
    .batch(batch_size=batch_size)
    .map(lambda x, y: (patch_extract_wrapper(x), y), num_parallel_calls=tf.data.AUTOTUNE)
    .prefetch(tf.data.AUTOTUNE)
)

# Create the validation dataset pipeline
dataset_val = (
    tf.data.Dataset.from_tensor_slices((x_val_split, y_val_split))
    .batch(batch_size=batch_size)
    .map(lambda x, y: (patch_extract_wrapper(x), y), num_parallel_calls=tf.data.AUTOTUNE)
    .prefetch(tf.data.AUTOTUNE)
)

print("Training and validation tf.data pipelines created.")

Training and validation tf.data pipelines created.


### Compile and Train the Model

Now, everything is ready. I'll compile the model with the AdamW optimizer and categorical cross-entropy loss. Then, I will start the first training run.

### Update: Resolving a Memory Error

After running the initial setup, the training failed due to a `ResourceExhaustedError`. Even with Google Colab Pro, the `batch_size` of 128 proved to be too demanding for the GPU. To fix this, I'm reducing the batch size to a more reasonable value of 64. I will now restart the training.

In [10]:
# Compile the model
model.compile(
    loss=keras.losses.CategoricalCrossentropy(label_smoothing=label_smoothing),
    optimizer=keras.optimizers.AdamW(
        learning_rate=learning_rate, weight_decay=weight_decay
    ),
    metrics=[
        keras.metrics.CategoricalAccuracy(name="accuracy"),
        keras.metrics.TopKCategoricalAccuracy(2, name="top-2-accuracy"),
    ],
)

# Start training
print("\nStarting the first training experiment...")
history = model.fit(
    dataset,
    batch_size=batch_size,
    epochs=num_epochs,
    validation_data=dataset_val,
)


Starting the first training experiment...
Epoch 1/150
124/124 ━━━━━━━━━━━━━━━━━━━━ 71s 394ms/step - accuracy: 0.4366 - loss: 1.6350 - top-2-accuracy: 0.6984 - val_accuracy: 0.6564 - val_loss: 1.1385 - val_top-2-accuracy: 0.8180
Epoch 2/150
124/124 ━━━━━━━━━━━━━━━━━━━━ 36s 287ms/step - accuracy: 0.6291 - loss: 1.1702 - top-2-accuracy: 0.7836 - val_accuracy: 0.6655 - val_loss: 1.0978 - val_top-2-accuracy: 0.8191
Epoch 3/150
124/124 ━━━━━━━━━━━━━━━━━━━━ 35s 277ms/step - accuracy: 0.6552 - loss: 1.1073 - top-2-accuracy: 0.8119 - val_accuracy: 0.6803 - val_loss: 1.0738 - val_top-2-accuracy: 0.8237
Epoch 4/150
124/124 ━━━━━━━━━━━━━━━━━━━━ 35s 282ms/step - accuracy: 0.6696 - loss: 1.0780 - top-2-accuracy: 0.8221 - val_accuracy: 0.6860 - val_loss: 1.0707 - val_top-2-accuracy: 0.8225
Epoch 5/150
124/124 ━━━━━━━━━━━━━━━━━━━━ 35s 281ms/step - accuracy: 0.6724 - loss: 1.0722 - top-2-accuracy: 0.8246 - val_accuracy: 0.6758 - val_loss: 1.0601 - val_top-2-accuracy: 0.8259
Epoch 6/150
124/124 ━━━━━━━

### Part 3: Analyzing the First Training Results

The training has now completed, and the results are not ideal. As can be seen from the logs above, the training accuracy climbed to over 97%, but the validation accuracy stalled around 68% and then started to fluctuate or even decrease. Similarly, the validation loss began to increase after an initial drop.

This is a clear indication of **severe overfiting**. The model has memorized the training set but failed to generalize to new data. For the next attempt, I need to implement a mechanism to stop the training process before this overfitting becomes too extreme.

### Part 4: Visualizing Performance and Adding Evaluation Metrics

To get a clearer picture of the overfitting problem, I'll now visualize the training history. I'm also adding more detailed evaluation functions to generate classification reports and confusion matrices for future experiments.

In [1]:

from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns

def plot_training_history(history):
    # Function to plot the training and validation curves
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    epochs = range(1, len(acc) + 1)

    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.plot(epochs, acc, 'bo-', label='Training Accuracy')
    plt.plot(epochs, val_acc, 'ro-', label='Validation Accuracy')
    plt.title('Training and Validation Accuracy')
    plt.legend()
    plt.grid(True)

    plt.subplot(1, 2, 2)
    plt.plot(epochs, loss, 'bo-', label='Training Loss')
    plt.plot(epochs, val_loss, 'ro-', label='Validation Loss')
    plt.title('Training and Validation Loss')
    plt.legend()
    plt.grid(True)
    plt.show()

# Plot the curves from the completed training run
print("Plotting the training history from the first experiment...")
plot_training_history(history)



def evaluate_model_on_dataset(model, dataset):
    print("\nGenerating predictions for evaluation...")
    y_pred_probs = model.predict(dataset)
    y_pred_classes = np.argmax(y_pred_probs, axis=1)

    y_true_classes = []
    for _, labels in dataset:
        y_true_classes.extend(np.argmax(labels.numpy(), axis=1))

    y_true_classes = np.array(y_true_classes)

    class_names = ['No DR', 'Mild', 'Moderate', 'Severe', 'Proliferative']

    # Classification Report
    print("\n--- Classification Report ---")
    print(classification_report(y_true_classes, y_pred_classes, target_names=class_names))

    # Confusion Matrix
    print("\n--- Confusion Matrix ---")
    cm = confusion_matrix(y_true_classes, y_pred_classes)
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
    plt.title('Confusion Matrix')
    plt.ylabel('Actual Label')
    plt.xlabel('Predicted Label')
    plt.show()

print("Evaluation functions defined.")

Plotting the training history from the first experiment...


NameError: name 'history' is not defined